![](../storage/banners/12_assumptions.png)

In [1]:
from __init__ import *

Инициализация проекта kiam-formation | Контекст: kiamformation


##### Проверка кватернионного умножения

###### Символьные выражения

In [3]:
from sympy import *

q1 = kf.get_vars("q^1", 4, numb=False)
q2 = kf.get_vars("q^2", 4, numb=False)
q1v = Matrix(q1[1:4])
q2v = Matrix(q2[1:4])

print("Скалярная часть:")
q1[0]*q2[0] - q1v.dot(q2v)

Скалярная часть:


q^1_0*q^2_0 - q^1_x*q^2_x - q^1_y*q^2_y - q^1_z*q^2_z

In [4]:
print("Векторная часть:")
q1v*q2[0] + q2v*q1[0] + q1v.cross(q2v)

Векторная часть:


Matrix([
[q^1_0*q^2_x + q^1_x*q^2_0 + q^1_y*q^2_z - q^1_z*q^2_y],
[q^1_0*q^2_y - q^1_x*q^2_z + q^1_y*q^2_0 + q^1_z*q^2_x],
[q^1_0*q^2_z + q^1_x*q^2_y - q^1_y*q^2_x + q^1_z*q^2_0]])

###### Старый код

In [100]:
import numpy as np

def vec2quat(v) -> list:
    if len(v) != 3:
        raise ValueError(f"Подаётся вектор длинны {len(v)}, требуется длинна 3!")
    if np.linalg.norm(v) > 1:
        return [0] + list(np.array(v)/np.linalg.norm(v))
    else:
        return [np.sqrt(1 - np.linalg.norm(v)**2), v[0], v[1], v[2]]

def q_dot(L1, L2):
    return np.array([L1[0] * L2[0] - L1[1] * L2[1] - L1[2] * L2[2] - L1[3] * L2[3],
                     L1[0] * L2[1] + L1[1] * L2[0] + L1[2] * L2[3] - L1[3] * L2[2],
                     L1[0] * L2[2] + L1[2] * L2[0] + L1[3] * L2[1] - L1[1] * L2[3],
                     L1[0] * L2[3] + L1[3] * L2[0] + L1[1] * L2[2] - L1[2] * L2[1]])

def quart2dcm(L) -> np.ndarray:
    w, x, y, z = L
    A = np.eye(3)
    A[0][0] = 1 - 2 * y ** 2 - 2 * z ** 2
    A[0][1] = 2 * x * y + 2 * z * w
    A[0][2] = 2 * x * z - 2 * y * w
    A[1][0] = 2 * x * y - 2 * z * w
    A[1][1] = 1 - 2 * x ** 2 - 2 * z ** 2
    A[1][2] = 2 * y * z + 2 * x * w
    A[2][0] = 2 * x * z + 2 * y * w
    A[2][1] = 2 * y * z - 2 * x * w
    A[2][2] = 1 - 2 * x ** 2 - 2 * y ** 2
    return A

def check_quat(q1, q2, q_dot, quart2dcm, vec2quat):
    print(f"λ₁ = {q1} \n(q3->q4: {vec2quat(q1[1:4])})")
    print(f"λ₂ = {q2}")
    print(f"λ₁∘λ₂ = {q_dot(q1, q2)}\n")
    print(f"A(λ₁) = \n{quart2dcm(q1)}")

qs = [np.random.rand(4) for _ in range(2)]
for i in range(2):
    qs[i] /= np.linalg.norm(qs)

check_quat(qs[0], qs[0], q_dot, quart2dcm, vec2quat)

λ₁ = [0.09424092 0.33736705 0.55605605 0.2465215 ] 
(q3->q4: [0.718479153601122, 0.3373670518828597, 0.5560560489710771, 0.24652149712988175])
λ₂ = [0.09424092 0.33736705 0.55605605 0.2465215 ]
λ₁∘λ₂ = [-0.47490635  0.06358756  0.10480647  0.04646483]

A(λ₁) = 
[[0.26005764 0.42165481 0.06152999]
 [0.32872515 0.65082125 0.3377471 ]
 [0.27114293 0.21057197 0.15397029]]


In [69]:
import quaternion
a = np.quaternion(0, 0, 1, 0)
b = np.quaternion(0, 1, 0, 0)
a * b

quaternion(0, 0, 0, -1)

###### Новый код

In [139]:
def vec2quat(v):
    """Перевод вектора кватерниона в кватернион"""
    if len(v) != 3:
        raise ValueError(f"Требуется длинна 3! (len(v)={len(v)})")
    if isinstance(v, np.ndarray | list):
        import quaternion
        
        print(f"Умное преобразование в кватернион! ({v}, тип: {type(v)}) | {isinstance(v, np.ndarray | list)}")
        return np.quaternion(*v)
    else:
        from sympy import Matrix
        return Matrix([sqrt(1 - v.dot(v)), v[0], v[1], v[2]])

def q_dot(q1, q2):
    if len(q1) == len(q2) == 4:
        from scipy.spatial.transform import Rotation
        print(f"Умное кватернионное умножение!")
        return (Rotation.from_quat(q1) * Rotation.from_quat(q2)).as_quat()
    else:
        from sympy import Matrix
        q1v = Matrix(q1[1:4])
        q2v = Matrix(q2[1:4])
        scalar = q1[0]*q2[0] - q1v.dot(q2v)
        vector = q1v*q2[0] + q2v*q1[0] + q1v.cross(q2v)
        return Matrix([scalar, vector[0], vector[1], vector[2]])
        '''return Matrix([q1[0] * q2[0] - q1[1] * q2[1] - q1[2] * q2[2] - q1[3] * q2[3],
                       q1[0] * q2[1] + q1[1] * q2[0] + q1[2] * q2[3] - q1[3] * q2[2],
                       q1[0] * q2[2] + q1[2] * q2[0] + q1[3] * q2[1] - q1[1] * q2[3],
                       q1[0] * q2[3] + q1[3] * q2[0] + q1[1] * q2[2] - q1[2] * q2[1]])'''

def quart2dcm(q):
    """Матрицу поворота из кватерниона поворота"""
    if isinstance(q, np.ndarray | list):
        from scipy.spatial.transform import Rotation
        print(f"Умное кватернионное преобразование в матрицу!")
        return Rotation.from_quat(q).as_matrix()
    else:
        from sympy import Matrix
        w, x, y, z = q
        return Matrix([[1 - 2*y**2 - 2*z**2, 2*x*y + 2*z*w, 2*x*z - 2*y*w],
                       [2*x*y - 2*z*w, 1 - 2*x**2 - 2*z**2, 2*y*z + 2*x*w],
                       [2*x*z + 2*y*w, 2*y*z - 2*x*w, 1 - 2*x**2 - 2*y**2]])

check_quat(qs[0], qs[0], q_dot, quart2dcm, vec2quat)

Умное преобразование в кватернион! ([0.33736705 0.55605605 0.2465215 ], тип: <class 'numpy.ndarray'>) | True
λ₁ = [0.09424092 0.33736705 0.55605605 0.2465215 ] 
(q3->q4: quaternion(0, 0.33736705188286, 0.556056048971077, 0.246521497129882))
λ₂ = [0.09424092 0.33736705 0.55605605 0.2465215 ]
Умное кватернионное умножение!
λ₁∘λ₂ = [ 0.09431245  0.33762311  0.5564781  -0.75329139]

Умное кватернионное преобразование в матрицу!
A(λ₁) = 
[[-0.71723737 -0.42741059  0.55035511]
 [ 0.6855456  -0.2912509   0.6672332 ]
 [-0.12489112  0.85585811  0.50190548]]


In [140]:
q1[1]

q^1_y

In [144]:
q1 = Matrix(kf.get_vars("q^1", 3, numb=False))
anw = vec2quat(q1)
anw

Matrix([
[sqrt(-q^1_x**2 - q^1_y**2 - q^1_z**2 + 1)],
[                                    q^1_x],
[                                    q^1_y],
[                                    q^1_z]])

In [145]:
anw.dot(anw)

1

In [131]:
print(f"({q1}, тип: {type(v)}) | {isinstance(q1, np.ndarray | list)}")

NameError: name 'v' is not defined